# Annex: Tabular data analysis with `pandas`

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install `pandas` and `numpy` by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install -q pandas numpy
```
:::

[Pandas](http://pandas.pydata.org/) is a an open source library providing tabular data structures and data analysis tools.In other words, if you can imagine the data in an Excel spreadsheet, then Pandas is the tool for the job.

<img src="https://media.geeksforgeeks.org/wp-content/cdn-uploads/creating_dataframe1.png" width="720px" />

:::{note}
Documentation for this package is available at https://pandas.pydata.org/docs/.
:::

## Package Imports

This will be our first experience with _importing_ a package.

Usually we import `pandas` with the _alias_ `pd`.

We might also need `numpy`, Python's main library for numerical computations.

In [ ]:
import pandas as pd
import numpy as np

## Series

A Series represents a one-dimensional array of data. It is similar to a dictionary consisting of an **index** and **values**, but has more functions. 

:::{note}
Example data on Germany's final six nuclear power plants is from [Wikipedia](https://en.wikipedia.org/wiki/List_of_power_stations_in_Germany#Nuclear).
:::

In [ ]:
names = ["Neckarwestheim", "Isar 2", "Emsland"]
values = [1269, 1365, 1290]
s = pd.Series(values, index=names)
s

In [ ]:
dictionary = {
    "Neckarwestheim": 1269,
    "Isar 2": 1365,
    "Emsland": 1290,
}
s = pd.Series(dictionary)
s

Arithmetic operations can be applied to the whole `pd.Series`.

In [ ]:
s**0.5

We can access the underlying index object if we need to:

In [ ]:
s.index

We can get values back out using the index via the `.loc` attribute

In [ ]:
s.loc["Isar 2"]

Or by raw position using `.iloc`

In [ ]:
s.iloc[2]

We can pass a list or array to loc to get multiple rows back:

In [ ]:
s.loc[["Neckarwestheim", "Emsland"]]

## DataFrame

Series are limited to a single **column**. A more useful Pandas data structure is the **DataFrame**. A DataFrame is basically a bunch of series that share the same index.

In [ ]:
data = {
    "capacity": [1269, 1365, 1290],  # MW
    "type": ["PWR", "PWR", "PWR"],
    "start_year": [1989, 1988, 1988],
    "end_year": [np.nan, np.nan, np.nan],
}
df = pd.DataFrame(data, index=["Neckarwestheim", "Isar 2", "Emsland"])
df

A wide range of statistical functions are available on both Series and DataFrames.

In [ ]:
df.min()

In [ ]:
df.mean(numeric_only=True)

We can get a single column as a Series using python's getitem syntax on the DataFrame object.

In [ ]:
df["capacity"]

Indexing works very similar to series

In [ ]:
df.loc["Emsland"]

But we can also specify the column(s) and row(s) we want to access

In [ ]:
df.at["Emsland", "start_year"]

We can also add new columns to the DataFrame:

In [ ]:
df["reduced_capacity"] = df.capacity * 0.8
df

We can also remove columns or rows from a DataFrame:

:::{note}
This operation needs to be an **inplace** operation to be permanent.
:::

In [ ]:
df.drop("reduced_capacity", axis="columns", inplace=True)

We can also drop columns with only NaN values

In [ ]:
df.dropna(axis=1)

Or fill it up with default "fallback" data:

In [ ]:
df.fillna(2023)

## Sorting Data

We can also sort the entries in dataframes, e.g. alphabetically by index or numerically by column values

In [ ]:
df.sort_index()

In [ ]:
df.sort_values(by="capacity", ascending=False)

## Filtering Data

We can also filter a DataFrame using a boolean series obtained from a condition. This is very useful to build subsets of the DataFrame.

In [ ]:
df.capacity > 1300

In [ ]:
df[df.capacity > 1300]

We can also combine multiple conditions, but we need to wrap the conditions with brackets!

In [ ]:
df[(df.capacity > 1300) & (df.start_year >= 1988)]

Or we make [SQL-like](https://en.wikipedia.org/wiki/SQL) queries:

In [ ]:
df.query("start_year == 1988")

In [ ]:
threshold = 1300
df.query("start_year == 1988 and capacity > @threshold")

## Modifying Values

In many cases, we want to modify values in a dataframe based on some rule. To modify values, we need to use `.loc` or `.iloc`

In [ ]:
df.loc["Isar 2", "capacity"] = 1366
df

Sometimes it can be useful to rename columns:

In [ ]:
df.rename(columns=dict(type="reactor"))

Sometimes it can be useful to replace values:

In [ ]:
df.replace({"PWR": "Pressurized water reactor"})

## Time Series

Time indexes are great when handling time-dependent data.

Let's first read some time series data, using the `pd.read_csv()` function, which takes a local file path ora link to an online resource.

The example data hourly time series for Germany in 2015 for:
    
1. electricity demand from [OPSD](https://open-power-system-data.org/) in GW
2. onshore wind capacity factors from [renewables.ninja](https://www.renewables.ninja/) in per-unit of installed capacity
3. offshore wind capacity factors from [renewables.ninja](https://www.renewables.ninja/) in per-unit of installed capacity
4. solar PV capacity factors from [renewables.ninja](https://www.renewables.ninja/) in per-unit of installed capacity
5. electricity day-ahead spot market prices in €/MWh from EPEX Spot zone DE/AT/LU retrieved via [SMARD platform](https://www.smard.de/home)

In [ ]:
url = (
    "https://tubcloud.tu-berlin.de/s/pKttFadrbTKSJKF/download/time-series-lecture-2.csv"
)
ts = pd.read_csv(url, index_col=0, parse_dates=True)
ts.head()

We can use Python's _slicing_ notation inside `.loc` to select a date range, and then use the built-in plotting feature of Pandas:

In [ ]:
ts.loc["2015-01-01":"2015-03-01", "load"].plot()

In [ ]:
ts.loc["2015-05-01", "solar"].plot()

A common operation is to change the resolution of a dataset by resampling in time, which Pandas exposes through the [resample](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#resampling) function.

:::{note}
The resample periods are specified using pandas [offset index](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases) syntax.
:::

In [ ]:
ts["onwind"].resample("ME").mean().plot()

## Groupby Functionality

`DataFrame` objects have a `groupby` method. The simplest way to think about it is that you pass another series, whose values are used to split the original object into different groups.

Here's an example which retrieves the total generation capacity per country:

In [ ]:
fn = "https://raw.githubusercontent.com/PyPSA/powerplantmatching/master/powerplants.csv"

In [ ]:
df = pd.read_csv(fn, index_col=0)
df.iloc[:5, :10]

In [ ]:
grouped = df.groupby("Country").Capacity.sum()
grouped.head()

Let's break apart this operation a bit. The workflow with `groupby` can be divided into three general steps:

1. **Split**: Partition the data into different groups based on some criterion.
2. **Apply**: Do some caclulation within each group, e.g. minimum, maximum, sums.
3. **Combine**: Put the results back together into a single object.

<img src="https://miro.medium.com/max/1840/1*JbF6nhrQsn4f-TaSF6IR9g.png" width="720px" />

Grouping is not only possible on a single columns, but also on multiple columns. For instance,
we might want to group the capacities by country **and** fuel type. To achieve this, we pass a list of functions to the `groupby` functions.

In [ ]:
capacities = df.groupby(["Country", "Fueltype"]).Capacity.sum()
capacities

By grouping by multiple attributes, our index becomes a `pd.MultiIndex` (a hierarchical index with multiple *levels*.

In [ ]:
capacities.index[:5]

We can use the `.unstack` function to reshape the multi-indexed `pd.Series` into a `pd.DataFrame` which has the second index level as columns. 

In [ ]:
capacities.unstack().tail().T

## Exercises

**Task 1:** Provide a list of unique fuel types included in the power plants dataset.

In [ ]:
df.Fueltype.unique()

**Task 2:** Filter the dataset by power plants with the fuel type "Hard Coal". How many hard coal power plants are there?

In [ ]:
coal = df.loc[df.Fueltype == "Hard Coal"]
coal

**Task 3:** Identify the three largest coal power plants. In which countries are they located? When were they built?

In [ ]:
coal.loc[coal.Capacity.nlargest(3).index]

**Task 4:** What is the average "DateIn" of each "Fueltype"? Which type of power plants is the oldest on average?

In [ ]:
2024 - df.groupby("Fueltype").DateIn.mean().sort_values()

**Task 5:** In the time series provided, calculate the annual average capacity factors of wind and solar.

In [ ]:
ts.mean()

**Task 6:** In the time series provided, calculate and plot the monthly average electricity price.

In [ ]:
ts["prices"].resample("ME").mean().plot()